In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

<b>Easy</b>
Взять один файл с маркой машины.

Построить прогноз, используя модели линейной регрессии и случайного леса. Сравнить результаты, используя кросс-валидацию. Подобрать гиперпараметры моделей (для линейной регрессии можно использовать реализацию с градиентным спуском SGDRegressor). Оценить качество, используя метрику r2_score. Оценить важность признаков (параметр feature_importance_ у случайного леса).

In [6]:
df = pd.read_csv('cars/audi.csv', parse_dates=True)
df.dropna()
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0
...,...,...,...,...,...,...,...,...,...
10663,A3,2020,16999,Manual,4018,Petrol,145,49.6,1.0
10664,A3,2020,16999,Manual,1978,Petrol,150,49.6,1.0
10665,A3,2020,17199,Manual,609,Petrol,150,49.6,1.0
10666,Q3,2017,19499,Automatic,8646,Petrol,150,47.9,1.4


In [7]:
df1 = pd.get_dummies(df, columns=['transmission','fuelType'])
df1


,model,year,price,mileage,tax,mpg,engineSize,transmission_Automatic,transmission_Manual,transmission_Semi-Auto,fuelType_Diesel,fuelType_Hybrid,fuelType_Petrol
0,A1,2017,12500,15735,150,55.4,1.4,0,1,0,0,0,1
1,A6,2016,16500,36203,20,64.2,2.0,1,0,0,1,0,0
2,A1,2016,11000,29946,30,55.4,1.4,0,1,0,0,0,1
3,A4,2017,16800,25952,145,67.3,2.0,1,0,0,1,0,0
4,A3,2019,17300,1998,145,49.6,1.0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10663,A3,2020,16999,4018,145,49.6,1.0,0,1,0,0,0,1
10664,A3,2020,16999,1978,150,49.6,1.0,0,1,0,0,0,1
10665,A3,2020,17199,609,150,49.6,1.0,0,1,0,0,0,1
10666,Q3,2017,19499,8646,150,47.9,1.4,1,0,0,0,0,1


In [8]:
x_df = df1[['year', 'mileage','tax','mpg','engineSize','transmission_Automatic','transmission_Manual', 'transmission_Semi-Auto', 'fuelType_Diesel','fuelType_Hybrid','fuelType_Petrol']]
x_df

,year,mileage,tax,mpg,engineSize,transmission_Automatic,transmission_Manual,transmission_Semi-Auto,fuelType_Diesel,fuelType_Hybrid,fuelType_Petrol
0,2017,15735,150,55.4,1.4,0,1,0,0,0,1
1,2016,36203,20,64.2,2.0,1,0,0,1,0,0
2,2016,29946,30,55.4,1.4,0,1,0,0,0,1
3,2017,25952,145,67.3,2.0,1,0,0,1,0,0
4,2019,1998,145,49.6,1.0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
10663,2020,4018,145,49.6,1.0,0,1,0,0,0,1
10664,2020,1978,150,49.6,1.0,0,1,0,0,0,1
10665,2020,609,150,49.6,1.0,0,1,0,0,0,1
10666,2017,8646,150,47.9,1.4,1,0,0,0,0,1


In [9]:
y_df = df1[['price']]
y_df

,price
0,12500
1,16500
2,11000
3,16800
4,17300
...,...
10663,16999
10664,16999
10665,17199
10666,19499


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.model_selection import train_test_split

In [11]:
x = x_df
y = y_df
#y = np.ravel(y_df)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
x_train.shape, x_test.shape

((8534, 11), (2134, 11))

<b>Модель линейной регрессии</b> 
** Модель SGDRegressor дала очень странные результаты, поэтому использовала LinearRegression

In [13]:
model_1 = LinearRegression()
model_1.fit(x_train, y_train)

y1_pred = model_1.predict(x_test)
r2_score(y_test, y1_pred)

0.8072929804794255

<b>Модель случайного леса

In [14]:
model_2 = RandomForestRegressor()
model_2.fit(x_train, y_train)

y2_pred = model_2.predict(x_test)
r2_score(y_test, y2_pred)

0.9473948920573461

In [15]:
from sklearn.model_selection import cross_val_score
cross_val1 = cross_val_score(model_1, x_train, y_train)
result_cross1 = cross_val1.mean()
cross_val2 = cross_val_score(model_2, x_train, y_train)
result_cross2 = cross_val2.mean()

In [16]:
print (cross_val1)
print (result_cross1)
print (cross_val2)
print (result_cross2)

[0.79080515 0.81219821 0.81596122 0.76715577 0.81248488]
0.7997210449923562
[0.94327327 0.9351806  0.93576513 0.93608495 0.95281302]
0.940623394064367


In [17]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [18]:
parametrs1 = { 'fit_intercept': [True, False],
              'n_jobs': range (1,2,3),
             }

In [19]:
crit1 = GridSearchCV(LinearRegression(), parametrs1, cv=5)
crit1.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False],
                         'n_jobs': range(1, 2, 3)})

In [20]:
crit1.best_params_

{'fit_intercept': False, 'n_jobs': 1}

In [21]:
parametrs2 = { 'n_estimators': range (10, 15, 50),
              'max_features': ["auto", "sqrt", "log2"],
              'min_samples_split': range (2,4,8)
            }
              

In [22]:
crit2 = GridSearchCV(RandomForestRegressor(), parametrs2, cv=5)
crit2.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': range(2, 4, 8),
                         'n_estimators': range(10, 15, 50)})

In [23]:
crit2.best_params_

{'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 10}

<b> Запуск моделей с подобранными параметрами

In [24]:
model_3 = LinearRegression(fit_intercept=False, n_jobs=1)
model_3.fit(x_train, y_train)
y3_pred = model_3.predict(x_test)
r2_score(y_test, y3_pred)

0.8072929804794234

In [25]:
model_4 = RandomForestRegressor(max_features='auto', min_samples_split = 2, n_estimators=10 )
model_4.fit(x_train, y_train)

y4_pred = model_4.predict(x_test)
r2_score(y_test, y4_pred)

0.9430484324142273

## Normal

Объединить в один датафрейм данные по всем маркам машин. Преобразовать категориальные признаки.

Построить еще несколько моделей, используя подбор гиперпараметров. Сравнить между собой все построенные модели.


In [26]:
df_all = pd.concat(map(pd.read_csv, ['cars/audi.csv', 'cars/bmw.csv','cars/cclass.csv', 'cars/focus.csv','cars/ford.csv']))
#я объединила только пять файлов, потому что иначе очень долго выполнялся код

In [27]:
df_all

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0
...,...,...,...,...,...,...,...,...,...
17960,Fiesta,2016,7999,Manual,31348,Petrol,125.0,54.3,1.2
17961,B-MAX,2017,8999,Manual,16700,Petrol,150.0,47.1,1.4
17962,B-MAX,2014,7499,Manual,40700,Petrol,30.0,57.7,1.0
17963,Focus,2015,9999,Manual,7010,Diesel,20.0,67.3,1.6


In [28]:
dff = df_all.dropna()
dff

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0
...,...,...,...,...,...,...,...,...,...
17960,Fiesta,2016,7999,Manual,31348,Petrol,125.0,54.3,1.2
17961,B-MAX,2017,8999,Manual,16700,Petrol,150.0,47.1,1.4
17962,B-MAX,2014,7499,Manual,40700,Petrol,30.0,57.7,1.0
17963,Focus,2015,9999,Manual,7010,Diesel,20.0,67.3,1.6


In [29]:
dff_1 = pd.get_dummies(dff, columns=['transmission','fuelType'])
dff_1

,model,year,price,mileage,tax,mpg,engineSize,transmission_Automatic,transmission_Manual,transmission_Semi-Auto,fuelType_Diesel,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,A1,2017,12500,15735,150.0,55.4,1.4,0,1,0,0,0,0,0,1
1,A6,2016,16500,36203,20.0,64.2,2.0,1,0,0,1,0,0,0,0
2,A1,2016,11000,29946,30.0,55.4,1.4,0,1,0,0,0,0,0,1
3,A4,2017,16800,25952,145.0,67.3,2.0,1,0,0,1,0,0,0,0
4,A3,2019,17300,1998,145.0,49.6,1.0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17960,Fiesta,2016,7999,31348,125.0,54.3,1.2,0,1,0,0,0,0,0,1
17961,B-MAX,2017,8999,16700,150.0,47.1,1.4,0,1,0,0,0,0,0,1
17962,B-MAX,2014,7499,40700,30.0,57.7,1.0,0,1,0,0,0,0,0,1
17963,Focus,2015,9999,7010,20.0,67.3,1.6,0,1,0,1,0,0,0,0


In [30]:
x1_df = dff_1[['year', 'mileage','tax','mpg','engineSize','transmission_Automatic','transmission_Manual', 'transmission_Semi-Auto', 'fuelType_Diesel','fuelType_Hybrid','fuelType_Petrol']]
y1_df = dff_1[['price']]

In [31]:
x = x1_df
y = y1_df


In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
x_train.shape, x_test.shape

((31531, 11), (7883, 11))

<b> RandomForestRegressor

In [39]:
parametrs3 = { 'n_estimators': range (10, 15, 50),
              'max_features': ["auto", "sqrt", "log2"],
              'min_samples_split': range (2,4,8)
            }
crit3 = RandomizedSearchCV(RandomForestRegressor(), parametrs3, cv=5)
crit3.fit(x_train, y_train)              
crit3.best_params_

{'n_estimators': 10, 'min_samples_split': 2, 'max_features': 'auto'}

In [40]:
model_forest = RandomForestRegressor(max_features='auto', min_samples_split = 2, n_estimators=10)
model_forest.fit(x_train, y_train)
yf_pred = model_forest.predict(x_test)
r2_score(y_test, yf_pred)

0.9266104729317104

<b> LinearRegression

In [41]:
from sklearn.linear_model import LinearRegression

In [51]:

parametrs4 = { 'fit_intercept': [True, False],
              'n_jobs': range (1,2,3),
             }
crit4 = RandomizedSearchCV(LinearRegression(), parametrs4)
crit4.fit(x_train, y_train)              
crit4.best_params_

{'n_jobs': 1, 'fit_intercept': False}

In [52]:
model_lin = LinearRegression(n_jobs=1, fit_intercept=False)
model_lin.fit(x_train, y_train)
yl_pred = model_lin.predict(x_test)
r2_score(y_test, yl_pred)

0.7408051762317249

In [54]:
a1 = r2_score(y_test, yf_pred)
a2 = r2_score(y_test, yl_pred)
if a1 > a2:
    print ('Результат модели RandomForestRegressor лучше')
else:
    print ('Результат модели LinearRegression лучше')

Результат модели RandomForestRegressor лучше
